# 金融数据分析和处理实战案例

### 1. 一些简单的金融应用和分析

In [20]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import tushare as ts
import warnings; warnings.simplefilter('ignore') #忽略可能会出现的警告信息，警告并不是错误，可以忽略；

In [2]:
hs300 = ts.get_k_data('hs300',start ='2015-01-01', end = '2017-06-30')

NameError: name 'ts' is not defined

In [ ]:
hs300.set_index('date', inplace = True)
hs300.head()

In [ ]:
hs300['close'].plot(figsize=(8, 5), grid=True, title = 'HS300 Close Price')    #必背；

In [ ]:
hs300['return'] = np.log(hs300['close'] / hs300['close'].shift(1))       #必背；SMA；连续收益的计算方法；

In [ ]:
hs300[['close', 'return']].tail()

In [ ]:
hs300[['close', 'return']].plot(subplots=True,
                              figsize=(10, 8))

In [ ]:
# 计算股价的移动平均，必背；SMA策略；      .rolling(window=).mean()
hs300['SMA20'] = hs300['close'].rolling(window=20).mean()
hs300.tail()

In [ ]:
# 用talib计算移动平均；talib：技术分析package；
import talib as ta
hs300['SMA20_ta'] = ta.SMA(np.asarray(hs300['close']),20)      #talib的数据结构有些指标只支持Ndarray格式；
hs300.tail()

In [ ]:
hs300.head()

In [ ]:
hs300[['close','SMA20']].plot(figsize = (8,6))

In [ ]:
hs300['SMA60'] = hs300['close'].rolling(window=60, min_periods=5).mean()
hs300.head()

In [ ]:
hs300[['close','SMA20','SMA60']].plot(figsize = (8,6))

In [ ]:
hs300['Upper'],hs300['middle'],hs300['Lower'] = ta.BBANDS(np.asarray(hs300['close']), timeperiod = 20, nbdevup = 2 , nbdevdn = 2, matype = 0) 
hs300[['close','Upper','middle','Lower']].plot(figsize = (10,6))

In [ ]:
import math
hs300['Mov_Vol'] = hs300['return'].rolling(window=252, min_periods = 60).std() * math.sqrt(252)

In [ ]:
hs300[['close', 'Mov_Vol', 'return']].plot(subplots=True,figsize=(10,8), grid=True)

### 2. 爬取Tushare数据并做选股条件分析

In [4]:
import tushare as ts
# 获取沪深300股票代码列表
hs300 = ts.get_hs300s()['code'].tolist()        #tolist()：把数据转成list；
hs300[:10]

['601611',
 '300024',
 '601919',
 '002195',
 '600150',
 '000060',
 '601800',
 '600570',
 '000778',
 '002292']

In [5]:
type(hs300)

list

In [7]:
# 获取基本面数据
# rev: 收入同比（%）  profit：利润同比（%）  npr: 净利润率（%）
stock_basics = ts.get_stock_basics()
stock_basics.head()

name industry area     pe  outstanding  totals  totalAssets  \
code                                                                  
002892  N科力尔     电气设备   湖南  29.21         0.18    0.84     45364.44   
603776  N永安行     公共交通   江苏  30.24         0.24    0.96    151001.23   
601326  秦港股份       港口   河北  16.80         5.58   55.87   2604904.25   
002175  东方网络     电器仪表   广西  66.28         5.18    7.54    311620.25   
002194  武汉凡谷     通信设备   湖北   0.00         5.22    5.65    243919.77   

        liquidAssets  fixedAssets   reserved   ...     bvps    pb  \
code                                           ...                  
002892      27348.35     16422.97   10559.05   ...     4.08  6.20   
603776      80017.61     13221.05   18186.32   ...     8.35  4.63   
601326     378642.06   1140481.38  450637.81   ...     2.40  1.55   
002175     129172.56     14333.55   52518.11   ...     1.98  5.65   
002194     178515.31     45737.78   82533.31   ...     3.12  2.84   

        timeToMarket       undp  perundp    rev   profit    gpr    npr  \
code                                                                     
002892      20170817    8897.21     1.06   0.00     0.00  28.04  12.92   
603776      20170817   32058.58     3.34   0.00     0.00  28.31  12.93   
601326      20170816  144972.61     0.26   0.00     0.00  38.15  18.18   
002175      20071012   19157.17     0.25  71.40   885.07  54.90  16.63   
002194      20071207   17823.45     0.32 -13.15 -4962.80 -10.51 -29.61   

        holders  
code             
002892    37145  
603776    25199  
601326   423429  
002175    66546  
002194    35108  

[5 rows x 22 columns]

In [8]:
stock_basics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3355 entries, 002892 to 603500
Data columns (total 22 columns):
name                3355 non-null object
industry            3351 non-null object
area                3351 non-null object
pe                  3355 non-null float64
outstanding         3355 non-null float64
totals              3355 non-null float64
totalAssets         3355 non-null float64
liquidAssets        3355 non-null float64
fixedAssets         3355 non-null float64
reserved            3355 non-null float64
reservedPerShare    3355 non-null float64
esp                 3355 non-null float64
bvps                3355 non-null float64
pb                  3355 non-null float64
timeToMarket        3355 non-null int64
undp                3355 non-null float64
perundp             3355 non-null float64
rev                 3355 non-null float64
profit              3355 non-null float64
gpr                 3355 non-null float64
npr                 3355 non-null float64
holders       

In [10]:
stock_basics.reset_index(inplace=True)      #reset_index重置dataFrame的索引
stock_basics.head()

index    code  name industry area     pe  outstanding  totals  totalAssets  \
0      0  002892  N科力尔     电气设备   湖南  29.21         0.18    0.84     45364.44   
1      1  603776  N永安行     公共交通   江苏  30.24         0.24    0.96    151001.23   
2      2  601326  秦港股份       港口   河北  16.80         5.58   55.87   2604904.25   
3      3  002175  东方网络     电器仪表   广西  66.28         5.18    7.54    311620.25   
4      4  002194  武汉凡谷     通信设备   湖北   0.00         5.22    5.65    243919.77   

   liquidAssets   ...     bvps    pb  timeToMarket       undp  perundp    rev  \
0      27348.35   ...     4.08  6.20      20170817    8897.21     1.06   0.00   
1      80017.61   ...     8.35  4.63      20170817   32058.58     3.34   0.00   
2     378642.06   ...     2.40  1.55      20170816  144972.61     0.26   0.00   
3     129172.56   ...     1.98  5.65      20071012   19157.17     0.25  71.40   
4     178515.31   ...     3.12  2.84      20071207   17823.45     0.32 -13.15   

    profit    gpr    npr  holders  
0     0.00  28.04  12.92    37145  
1     0.00  28.31  12.93    25199  
2     0.00  38.15  18.18   423429  
3   885.07  54.90  16.63    66546  
4 -4962.80 -10.51 -29.61    35108  

[5 rows x 24 columns]

In [11]:
#isin():数据过滤的方法；从stcok_basics这张大表格里面把hs300的这300只股票代码给挑选出来；
data1 = stock_basics.loc[stock_basics['code'].isin(hs300),        
                         ['code', 'name', 'industry', 'pe', 'pb', 'esp','rev', 'profit',]]
data1.head()

code  name industry      pe    pb    esp    rev  profit
29  601611  中国核建     建筑施工   72.49  3.68  0.042   4.92   21.65
36  300024   机器人     专用机械  100.50  6.23  0.113  15.47    6.67
56  601919  中远海控       水运   74.69  4.29  0.026  48.15  106.06
57  002195  二三四五     软件服务   34.33  4.25  0.064  11.50  123.87
60  600150  中国船舶       船舶    0.00  2.39 -0.052 -29.80 -162.17

In [13]:
data1.columns = ['代码','名称', '行业', 'PE', 'PB', 'EPS', '收入%', '利润%']
data1.head()

代码    名称    行业      PE    PB    EPS    收入%     利润%
29  601611  中国核建  建筑施工   72.49  3.68  0.042   4.92   21.65
36  300024   机器人  专用机械  100.50  6.23  0.113  15.47    6.67
56  601919  中远海控    水运   74.69  4.29  0.026  48.15  106.06
57  002195  二三四五  软件服务   34.33  4.25  0.064  11.50  123.87
60  600150  中国船舶    船舶    0.00  2.39 -0.052 -29.80 -162.17

In [14]:
# 获取盈利能力数据
stock_profit = ts.get_profit_data(2017,1)
stock_profit.head()

[Getting data:]#######################################################

code   name     roe  net_profit_ratio  gross_profit_rate  net_profits  \
0  600423  *ST柳化  215.12             10.30            -1.3970      40.6863   
1  600306    商业城   71.97             71.48            17.3030     171.9346   
2  000717   韶钢松山   26.83              3.19             6.2247     168.1588   
3  600133   东湖高新   26.81             55.22            11.6396     686.8356   
4  600354   敦煌种业   23.40            302.21            25.0654     273.7035   

      eps  business_income    bips  
0  0.1018         394.6887  0.9883  
1  0.9651         240.5080  1.3501  
2  0.0695        5268.7636  2.1776  
3  1.0828        1243.7266  1.9609  
4  0.5185          90.5655  0.1715

In [15]:
data2 = stock_profit.loc[stock_profit['code'].isin(hs300), ['code', 'roe', 'gross_profit_rate', 'net_profit_ratio']]

In [17]:
data2.columns = ['代码', 'ROE', '毛利率', '净利率']
data2 = round(data2,2)
data2.head()

代码    ROE    毛利率    净利率
18  002027  12.20  67.60  45.72
21  600309  11.26  39.23  19.94
25  002714  10.72  40.47  34.55
34  002304   9.52  61.23  36.14
61  600519   7.74  91.16  46.00

In [18]:
# 获取成长能力数据
# nprg:净利润增长率（%）  nav:净资产增长率
stock_growth = ts.get_growth_data(2017,1)
data3 = stock_growth.loc[stock_growth['code'].isin(hs300), ['code', 'nprg']]
data3.columns = ['代码', 'NI%', ]
data3 = round(data3,2)
data3.head()

[Getting data:]#######################################################

代码      NI%
20  601225  3805.27
33  000725  2443.31
42  000778  1948.33
46  000425  1780.52
50  601600  1579.24

In [23]:
from functools import reduce

# 数据合并
merge = lambda x,y: pd.merge(x, y, how='left', on='代码')     #定义了merge函数；
data = reduce(merge, [data1, data2, data3])
data.drop_duplicates(inplace=True)
data.head()

代码    名称    行业      PE    PB    EPS    收入%     利润%   ROE    毛利率    净利率  \
0  601611  中国核建  建筑施工   72.49  3.68  0.042   4.92   21.65  1.26   8.40   1.37   
1  300024   机器人  专用机械  100.50  6.23  0.113  15.47    6.67  1.03  32.67  12.84   
2  601919  中远海控    水运   74.69  4.29  0.026  48.15  106.06  1.43   7.69   1.34   
3  002195  二三四五  软件服务   34.33  4.25  0.064  11.50  123.87  3.09  90.00  49.84   
4  600150  中国船舶    船舶    0.00  2.39 -0.052 -29.80 -162.17 -0.48  10.56  -1.75   

      NI%  
0   27.00  
1    3.41  
2     NaN  
3  124.06  
4 -319.53

In [21]:
hs300 = pd.merge(data1,data2, how='left', on='代码')
hs300.head()

代码    名称    行业      PE    PB    EPS    收入%     利润%   ROE    毛利率    净利率
0  601611  中国核建  建筑施工   72.49  3.68  0.042   4.92   21.65  1.26   8.40   1.37
1  300024   机器人  专用机械  100.50  6.23  0.113  15.47    6.67  1.03  32.67  12.84
2  601919  中远海控    水运   74.69  4.29  0.026  48.15  106.06  1.43   7.69   1.34
3  002195  二三四五  软件服务   34.33  4.25  0.064  11.50  123.87  3.09  90.00  49.84
4  600150  中国船舶    船舶    0.00  2.39 -0.052 -29.80 -162.17 -0.48  10.56  -1.75

In [22]:
hs300 = pd.merge(hs300,data3, how='left', on='代码')
hs300.drop_duplicates(inplace=True)             #去除重复数据；drop_duplicates()
hs300.head()

代码    名称    行业      PE    PB    EPS    收入%     利润%   ROE    毛利率    净利率  \
0  601611  中国核建  建筑施工   72.49  3.68  0.042   4.92   21.65  1.26   8.40   1.37   
1  300024   机器人  专用机械  100.50  6.23  0.113  15.47    6.67  1.03  32.67  12.84   
2  601919  中远海控    水运   74.69  4.29  0.026  48.15  106.06  1.43   7.69   1.34   
3  002195  二三四五  软件服务   34.33  4.25  0.064  11.50  123.87  3.09  90.00  49.84   
4  600150  中国船舶    船舶    0.00  2.39 -0.052 -29.80 -162.17 -0.48  10.56  -1.75   

      NI%  
0   27.00  
1    3.41  
2     NaN  
3  124.06  
4 -319.53

In [24]:
# 根据已有列计算新数据
data['估值系数'] = data['PE'] * data['PB']
data = round(data,2)
data.head(5)

代码    名称    行业      PE    PB   EPS    收入%     利润%   ROE    毛利率    净利率  \
0  601611  中国核建  建筑施工   72.49  3.68  0.04   4.92   21.65  1.26   8.40   1.37   
1  300024   机器人  专用机械  100.50  6.23  0.11  15.47    6.67  1.03  32.67  12.84   
2  601919  中远海控    水运   74.69  4.29  0.03  48.15  106.06  1.43   7.69   1.34   
3  002195  二三四五  软件服务   34.33  4.25  0.06  11.50  123.87  3.09  90.00  49.84   
4  600150  中国船舶    船舶    0.00  2.39 -0.05 -29.80 -162.17 -0.48  10.56  -1.75   

      NI%    估值系数  
0   27.00  266.76  
1    3.41  626.12  
2     NaN  320.42  
3  124.06  145.90  
4 -319.53    0.00

In [25]:
#条件选股；
data_filtered = data.loc[(data['估值系数'] < 60) & (data['ROE'] > 5),
                     ['代码', '名称', 'PE', 'PB', '估值系数', 'ROE', '收入%']]


In [26]:
data_filtered.head()

代码    名称     PE    PB   估值系数    ROE    收入%
14  600115  东方航空   9.04  2.03  18.35   5.62   4.26
33  603885  吉祥航空  15.95  3.33  53.11   5.22  22.02
44  600309  万华化学   9.71  4.49  43.60  11.26  95.82
45  002065  东华软件  17.57  1.81  31.80   5.22  21.61
76  600688  上海石化   9.35  2.70  25.24   7.22  36.26

In [27]:
print('筛选结果共 %d 只个股' % len(data_filtered))     #占位符格式化；format格式化；

筛选结果共 16 只个股


In [28]:
# 按选定字段对数据进行排序
data_filtered.sort_values(['估值系数'], ascending=True, inplace=True)    # sort_values具体实战应用；
data_filtered.head(10)

代码    名称     PE    PB   估值系数   ROE     收入%
125  000625  长安汽车   6.83  1.43   9.77  5.22   -8.64
274  600177   雅戈尔   7.21  1.58  11.39  5.47  -38.93
152  601225  陕西煤业   7.31  2.09  15.28  7.13  110.22
14   600115  东方航空   9.04  2.03  18.35  5.62    4.26
142  000540  中天金融   9.17  2.16  19.81  5.88    9.28
264  601318  中国平安  10.13  2.27  23.00  5.60   27.19
76   600688  上海石化   9.35  2.70  25.24  7.22   36.26
45   002065  东华软件  17.57  1.81  31.80  5.22   21.61
158  600690  青岛海尔  11.42  2.81  32.09  6.13   69.73
205  600340  华夏幸福  13.56  3.06  41.49  5.64   57.71

In [31]:
# 数据分类
def map_func(x):
    """
    作为 apply 函数的参数传入
    :param x: df 中一行或一列数据，取决于 apply 函数的参数 axis
    :return: 将每个计算结果组合，返回一个 series
    """
    if x['ROE'] > 5:
        return '高成长'
    elif x['ROE'] >= 0:
        return '低成长'
    elif x['ROE'] < 0:
        return '亏损'
    
# 根据 ROE 数据计算“成长性”
data['成长性'] = data.apply(map_func, axis=1)
data.tail(10)

代码     名称    行业     PE    PB   EPS    收入%    利润%   ROE    毛利率    净利率  \
299  300146   汤臣倍健    食品  18.73  4.47  0.40  20.53  62.29  6.18  64.94  38.47   
300  002310   东方园林  建筑施工  53.58  5.23  0.17  70.79  57.40 -0.38  32.11  -2.85   
301  000568   泸州老窖    白酒  24.58  6.64  0.57  20.34  34.54  6.75  70.25  29.86   
302  002008   大族激光  电器仪表  61.84  6.75  0.14  28.79  31.50  2.72  40.76   9.95   
303  000858  五 粮 液    白酒  14.74  4.18  0.95  15.11  23.77  7.09  70.53  35.36   
304  601601   中国太保    保险  38.12  2.30  0.22  29.91  -9.05  1.50   2.63   1.84   
305  601336   新华保险    保险  25.43  3.03  0.59 -19.31  -7.87  2.98   6.20   4.11   
306  002241   歌尔股份   元器件  55.82  5.61  0.10  68.33  51.60  2.66  22.33   6.60   
307  002475   立讯精密   元器件  56.49  5.88  0.10  87.57  51.17  2.60  20.84   7.65   
308  600373   中文传媒   出版业  18.35  3.04  0.34  -8.12  24.13  4.13  41.93  16.20   

       NI%    估值系数  成长性  
299  31.01   83.72  高成长  
300    NaN  280.22   亏损  
301  33.45  163.21  高成长  
302  33.59  417.42  低成长  
303  24.30   61.61  高成长  
304  -9.29   87.68  低成长  
305  -7.87   77.05  低成长  
306  48.26  313.15  低成长  
307  52.44  332.16  低成长  
308  23.93   55.78  低成长

In [34]:
# 对高成长分类按照‘烟蒂系数’做升序排列
# 选择具体的某一列，按照一定的顺序进行呈现；
data_growth = data[data['成长性'] == '高成长'].sort_values(['估值系数'], ascending=True)
data_growth.head(20)

代码     名称    行业     PE    PB   EPS     收入%      利润%    ROE    毛利率  \
125  000625   长安汽车  汽车整车   6.83  1.43  0.50   -8.64   -10.16   5.22  15.15   
274  600177    雅戈尔    服饰   7.21  1.58  0.35  -38.93   -48.45   5.47  46.77   
152  601225   陕西煤业  煤炭开采   7.31  2.09  0.27  110.22  6982.83   7.13  55.72   
14   600115   东方航空    空运   9.04  2.03  0.20    4.26     8.30   5.62  12.44   
142  000540   中天金融  多元金融   9.17  2.16  0.20    9.28    -5.63   5.88  39.99   
264  601318   中国平安    保险  10.13  2.27  1.29   27.19    11.37   5.60  12.29   
76   600688   上海石化  石油加工   9.35  2.70  0.18   36.26    68.42   7.22  28.62   
45   002065   东华软件  软件服务  17.57  1.81  0.15   21.61   168.77   5.22  32.28   
158  600690   青岛海尔  家用电器  11.42  2.81  0.28   69.73     8.89   6.13  30.10   
205  600340   华夏幸福  区域地产  13.56  3.06  0.59   57.71    35.06   5.64  62.39   
44   600309   万华化学  化工原料   9.71  4.49  1.79   95.82   255.35  11.26  39.23   
176  600415   小商品城   商品城  14.26  3.59  0.12  323.71   222.60   6.29  27.25   
33   603885   吉祥航空    空运  15.95  3.33  0.23   22.02    -7.45   5.22  21.33   
296  002304   洋河股份    白酒  12.11  4.61  1.82   10.90    11.68   9.52  61.23   
177  000651   格力电器  家用电器  14.27  3.96  0.67   20.46    27.05   6.93  34.37   
141  000333   美的集团  家用电器  14.66  3.88  0.67   55.85    11.40   6.64  25.06   
303  000858  五 粮 液    白酒  14.74  4.18  0.95   15.11    23.77   7.09  70.53   
253  002714   牧原股份  农业综合  15.47  4.07  1.12   85.21    11.92  10.72  40.47   
175  600816   安信信托  多元金融  18.93  4.33  0.35   14.39    15.73   7.55  83.30   
299  300146   汤臣倍健    食品  18.73  4.47  0.40   20.53    62.29   6.18  64.94   

       净利率      NI%   估值系数  成长性  
125  13.59   -10.13   9.77  高成长  
274  37.17   -50.95  11.39  高成长  
152  21.78  3805.27  15.28  高成长  
14   11.48     5.40  18.35  高成长  
142  15.42    -6.17  19.81  高成长  
264   8.67    10.05  23.00  高成长  
76    8.54    67.48  25.24  高成长  
45   36.45   173.18  31.80  高成长  
158   4.60    10.34  32.09  高成长  
205  22.47    43.97  41.49  高成长  
44   19.94   375.02  43.60  高成长  
176  14.45   224.82  51.19  高成长  
33   13.50    -4.41  53.11  高成长  
296  36.14    11.46  55.83  高成长  
177  13.52    27.07  56.51  高成长  
141   7.28    12.46  56.88  高成长  
303  35.36    24.30  61.61  高成长  
253  34.55    84.81  62.96  高成长  
175  62.47    67.73  81.97  高成长  
299  38.47    31.01  83.72  高成长

In [36]:
data_profit = data[data['成长性'] == '高成长'].sort_values(['ROE'], ascending=False)
data_profit.head()

代码    名称    行业     PE     PB   EPS    收入%     利润%    ROE    毛利率  \
285  002027  分众传媒  影视音像  25.58  12.49  0.09  14.99   70.42  12.20  67.60   
44   600309  万华化学  化工原料   9.71   4.49  1.79  95.82  255.35  11.26  39.23   
253  002714  牧原股份  农业综合  15.47   4.07  1.12  85.21   11.92  10.72  40.47   
296  002304  洋河股份    白酒  12.11   4.61  1.82  10.90   11.68   9.52  61.23   
292  600519  贵州茅台    白酒  27.51   8.18  8.96  33.11   27.81   7.74  91.16   

       净利率     NI%    估值系数  成长性  
285  45.72   71.73  319.49  高成长  
44   19.94  375.02   43.60  高成长  
253  34.55   84.81   62.96  高成长  
296  36.14   11.46   55.83  高成长  
292  46.00   25.57  225.03  高成长

In [40]:
def group_func(df):
    """
    作为 groupby.apply 函数的参数传入
    :param df: 实际是经过聚合后的单一类的 dataframe
    :return: 返回 df 按照’烟蒂系数‘排序后的前三位
    """
    return df.sort_values(['估值系数'], ascending=True)[:2]

# 按‘成长性’分组，筛选每个分组中‘烟蒂系数’最低的两个个股


In [37]:
data_grouped = data.groupby('成长性')

In [38]:
data_grouped.size()

成长性
亏损      18
低成长    244
高成长     37
dtype: int64

In [41]:
data_grouped = data.groupby('成长性').apply(group_func)
data_grouped

代码    名称    行业    PE    PB   EPS    收入%     利润%   ROE    毛利率  \
成长性                                                                         
亏损  4    600150  中国船舶    船舶  0.00  2.39 -0.05 -29.80 -162.17 -0.48  10.56   
    286  000768  中航飞机    航空  0.00  3.56 -0.03 -13.39  -47.23 -0.57   5.30   
低成长 159  600016  民生银行    银行  5.28  0.86  0.39  -9.73    3.60  3.97  48.48   
    106  601818  光大银行    银行  5.58  0.82  0.18   0.50    1.57  3.26  48.37   
高成长 125  000625  长安汽车  汽车整车  6.83  1.43  0.50  -8.64  -10.16  5.22  15.15   
    274  600177   雅戈尔    服饰  7.21  1.58  0.35 -38.93  -48.45  5.47  46.77   

           净利率     NI%   估值系数  成长性  
成长性                                 
亏损  4    -1.75 -319.53   0.00   亏损  
    286  -2.80     NaN   0.00   亏损  
低成长 159  39.19    3.99   4.54  低成长  
    106  36.26    1.64   4.58  低成长  
高成长 125  13.59  -10.13   9.77  高成长  
    274  37.17  -50.95  11.39  高成长

In [43]:
data_grouped = data.groupby('行业').apply(group_func)
data_grouped.head(10)

代码     名称    行业      PE    PB   EPS    收入%     利润%   ROE    毛利率  \
行业                                                                              
专用机械 46   601608   中信重工  专用机械    0.00  3.23 -0.00   4.14 -144.77 -0.03  19.87   
     1    300024    机器人  专用机械  100.50  6.23  0.11  15.47    6.67  1.03  32.67   
中成药  207  000623   吉林敖东   中成药   15.42  1.34  0.36 -11.73   15.12  2.17  67.75   
     204  600332    白云山   中成药   21.93  2.38  0.30   4.69   18.41  2.71  39.37   
乳制品  192  600887   伊利股份   乳制品   20.06  5.61  0.28   3.23   11.58  6.98  37.64   
互联网  189  603000    人民网   互联网    0.00  6.17 -0.04  -7.43 -216.53 -1.67  26.55   
     244  002131   利欧股份   互联网   28.30  2.26  0.03  51.50   12.14  1.99  18.09   
仓储物流 243  002183  怡 亚 通  仓储物流   22.97  2.99  0.09  15.04   13.34  3.27   6.41   
保险   264  601318   中国平安    保险   10.13  2.27  1.29  27.19   11.37  5.60  12.29   
     305  601336   新华保险    保险   25.43  3.03  0.59 -19.31   -7.87  2.98   6.20   

            净利率     NI%    估值系数  成长性  
行业                                    
专用机械 46   -0.21  -49.50    0.00   亏损  
     1    12.84    3.41  626.12  低成长  
中成药  207  83.47   14.63   20.66  低成长  
     204   9.16   18.82   52.19  低成长  
乳制品  192  11.03   11.53  112.54  高成长  
互联网  189 -16.72 -151.03    0.00   亏损  
     244   7.15   11.56   63.96  低成长  
仓储物流 243   1.20    2.07   68.68  低成长  
保险   264   8.67   10.05   23.00  高成长  
     305   4.11   -7.87   77.05  低成长

声明：本资料仅供内部学习交流使用，切勿外传。